# $\color{Green}{\text{Fundamentals of Inference and Learning}}$ 
## Homework 1 
### Exercise 1 : Statistical Inference and Maximum Likelihood

#### 1.
First we need to compute $Z(\lambda)$ in order to have a normalized distribution. We therefore have:
$$
\int_{-\infty}^{-\infty} p_\lambda (x_i)dx_i = 1 
\Leftrightarrow \int_{1}^{20} \frac{1}{Z(\lambda)}e^{\frac{-x_i}{\lambda}}dx_i = 1
$$
By integrating:
$$
-\frac{\lambda}{Z(\lambda)}*(e^{\frac{-20}{\lambda}}-e^{\frac{-1}{\lambda}}) = 1
$$
$Z(\lambda)$ is found to be : 
$$
    Z(\lambda) = -\lambda * (e^{\frac{-20}{\lambda}}-e^{\frac{-1}{\lambda}})
$$

Now compute $\mathbb{E}[X]$
$$
\mathbb{E}[X] = \int_{1}^{20} x_i \cdot \frac{1}{Z(\lambda)}e^{\frac{-x_i}{\lambda}}dx_i
$$
Let's integrate by parts:
$$
\mathbb{E}[X] = \frac{1}{Z(\lambda)} \{ -\lambda[20e^{-\frac{20}{\lambda}}-e^{-\frac{1}{\lambda}}] - \lambda \int_{1}^{20} e^{\frac{-x_i}{\lambda}}dx_i\}
$$
Finally:
$$
    \mathbb{E}[X] = \frac{1}{Z(\lambda)} \{ -\lambda[20e^{-\frac{20}{\lambda}}-e^{-\frac{1}{\lambda}}] + \lambda^2 [e^{-\frac{20}{\lambda}}-e^{-\frac{1}{\lambda}}]\}
$$ 

Now lets compute $\partial_\lambda Z(\lambda)$:
$$
\partial_\lambda Z(\lambda) = -\frac{1}{\lambda(e^{\frac{-20}{\lambda}}-e^{\frac{-1}{\lambda}})} [-(e^{-\frac{20}{\lambda}}-e^{-\frac{1}{\lambda}})-\lambda(\frac{20e^{-\frac{20}{\lambda}}}{\lambda^2}-\frac{-e^{-\frac{1}{\lambda}}}{\lambda^2}] = \frac{1}{Z(\lambda)} \{ -\lambda[20e^{-\frac{20}{\lambda}}-e^{-\frac{1}{\lambda}}] + \lambda^2 [e^{-\frac{20}{\lambda}}-e^{-\frac{1}{\lambda}}]\}
$$

By comparing the previous result found for $\mathbb{E}[X]$, the final expression is:
$$
\mathbb{E}[X] = \lambda^2 \cdot \partial_\lambda Z(\lambda)
$$